# IN_SYS - SW08 - XGBoost in Python for Heart Disease prediction, Homework 5
Adopted for IN_SYS from Joshua Starmer, StatQuest!

----

In this lesson we will use **XGBoost** to build a collection of boosted decision trees (one of which is illustrated below), and use continuous and categorical data from the **[UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php)** and **[Heart Disease Dataset](https://archive.ics.uci.edu/ml/datasets/Heart+Disease)** to predict whether or not a patient has a heart disease.  

Attention: this is **classification** problem.


#### Also Note:
I strongly encourage you to play around with the code. Playing with the code is the best way to learn from it.

-----

# Import the modules that will do all the work


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix

import xgboost as xgb

----

# Import the data
You can import the data from **ILIAS: heart_disease_data.csv**. This file is based on the original data from the **[UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php)** repository but is preprocessed in sense that missing data has been removed. This dataset will allow us to predict if someone has heart disease based on their sex, age, blood pressure and a variety of other metrics.

## Load and Display the data "heart_disease_data.csv"

,age,sex,cp,restbp,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,hd
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,1
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


## Description of column names

- **age**, **Float**
- **sex** - **Category**
  - 0 = female
  - 1 = male
- **cp**, chest pain, **Category**
  - 1 = typical angina
  - 2 = atypical angina
  - 3 = non-anginal pain
  - 4 = asymptomatic
- **restbp**, resting blood pressure (in mm Hg), **Float**
- **chol**, serum cholesterol in mg/dl, **Float**
- **fbs**, fasting blood sugar, **Category**
  - 0 = >=120 mg/dl
  - 1 = <120 mg/dl
- **restecg**, resting electrocardiographic results, **Category**
  - 1 = normal
  - 2 = having ST-T wave abnormality
  - 3 = showing probable or definite left ventricular hypertrophy
- **thalach**,  maximum heart rate achieved, **Float**
- **exang**, exercise induced angina, **Category**
  - 0 = no
  - 1 = yes
- **oldpeak**, ST depression induced by exercise relative to rest. **Float**
- **slope**, the slope of the peak exercise ST segment, **Category**
  - 1 = upsloping
  - 2 = flat
  - 3 = downsloping
- **ca**, number of major vessels (0-3) colored by fluoroscopy, **Float**
- **thal**, thalium heart scan, **Category**
  - 3 = normal (no cold spots)
  - 6 = fixed defect (cold spots during rest and exercise)
  - 7 = reversible defect (when cold spots only appear during exercise)
- **hd**, heart disease, **Category**
  - 0 = No
  - 1 = Yes


----

# Split the Data into Dependent and Independent Variables

We need to format the data for our **XGBoost** model.

The first step is to split the data into two parts:
1. The columns of data that we will use to make classifications
2. The column of data that we want to predict.

We will use the conventional notation of `X` (capital **X**) to represent the columns of data that we will use to make classifications and `y` (lower case **y**) to represent the thing we want to predict. In this case, we want to predict **hd** (heart disease).  

Please use **dataframe** type of data object as the target where you will store both X and y.  

In [3]:
# We need to input data in variable "X" (without column hd)


,age,sex,cp,restbp,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0


In [4]:
# We need the output column hd (Heart Disease yes or no) in variable y


0    0
1    1
2    1
3    0
4    0
Name: hd, dtype: int64

Now that we have created **X**, which has the data we want to use to make predictions, and **y**, which has the data we want to predict, we are ready ton continue formatting **X** so that it is suitable for making a model with **XGBoost**.

----

# One-Hot Encoding

Now that we have split the data frame into two pieces, `X`, which contains the data we will use to make, or predict, classifications, and `y`, which contains the known classifications in our training dataset, we need to take a closer look at the variables in `X`. The list below tells us what each variable represents and the type of data (**float** or **categorical**) it should contain:

- **age**, **Float**
- **sex** - **Category**
  - 0 = female
  - 1 = male
- **cp**, chest pain, **Category**
  - 1 = typical angina
  - 2 = atypical angina
  - 3 = non-anginal pain
  - 4 = asymptomatic
- **restbp**, resting blood pressure (in mm Hg), **Float**
- **chol**, serum cholesterol in mg/dl, **Float**
- **fbs**, fasting blood sugar, **Category**
  - 0 = >=120 mg/dl
  - 1 = <120 mg/dl
- **restecg**, resting electrocardiographic results, **Category**
  - 1 = normal
  - 2 = having ST-T wave abnormality
  - 3 = showing probable or definite left ventricular hypertrophy
- **thalach**,  maximum heart rate achieved, **Float**
- **exang**, exercise induced angina, **Category**
  - 0 = no
  - 1 = yes
- **oldpeak**, ST depression induced by exercise relative to rest. **Float**
- **slope**, the slope of the peak exercise ST segment, **Category**
  - 1 = upsloping
  - 2 = flat
  - 3 = downsloping
- **ca**, number of major vessels (0-3) colored by fluoroscopy, **Float**
- **thal**, thalium heart scan, **Category**
  - 3 = normal (no cold spots)
  - 6 = fixed defect (cold spots during rest and exercise)
  - 7 = reversible defect (when cold spots only appear during exercise)

Now, just to review, let's look at the data types in `X` to remember how python is seeing the data right now.

In [5]:
X.dtypes

age        float64
sex        float64
cp         float64
restbp     float64
chol       float64
fbs        float64
restecg    float64
thalach    float64
exang      float64
oldpeak    float64
slope      float64
ca         float64
thal       float64
dtype: object

**XGBoost** natively supports continuous data, like resting blood pressure (**restbp**) and maximum heart rate (**thalach**), it does not natively support categorical data, like chest pain (**cp**), which contains 4 different categories. Thus, in order to use categorical data with **XGBoost**, we have to use a trick that converts a column of categorical data into multiple columns of binary values. This trick is called **One-Hot Encoding**.

At this point you may be wondering, "what's wrong with treating categorical data like continuous data?" To answer that question, let's look at an example: For the **cp** (chest pain) column, we have 4 options:
1. typical angina
2. atypical angina
3. non-anginal pain
4. asymptomatic

If we treated these values, 1, 2, 3 and 4, like continuous data, then we would assume that 4, which means "asymptomatic", is more similar to 3, which means "non-anginal pain", than it is to 1 or 2, which are other types of chest pain. That means the **XGBoost Tree** would be more likely to cluster the patients with 4s and 3s together than the patients with 4s and 1s together. In contrast, if we treat these numbers like categorical data, then we treat each one a separate category that is no more or less similar to any of the other categories. Thus, the likelihood of clustering patients with 4s with 3s is the same as clustering 4s with 1s, and that approach is more reasonable.

Now let's inspect and, if needed, convert the columns that contain categorical and integer data into the correct datatypes. We'll start with **cp** (chest pain) by inspecting all of its unique values:
<!-- We'll start with the three colunms that should only contain 0s and 1s. **sex**. First, let's make sure it only contains `0` (for **female**) and `1` (for **male**). -->

In [6]:
X['cp'].unique()

array([1., 4., 3., 2.])

So, the good news is that **cp** only contains the values it is supposed to contain, so we will convert it, using **One-Hot Encoding**, into a series of columns that only contains **0s** and **1s**.

**NOTE:** There are many different ways to do **One-Hot Encoding** in Python. Two of the more popular methods are `ColumnTransformer()` (from **scikit-learn**) and `get_dummies()` (from **pandas**). For the sake of learning how **One-Hot Encoding** works, I prefer to use `get_dummies()`. However, once you are comfortable with **One-Hot Encoding**, I encourage you to investigate using `ColumnTransformer()`.

First, before we commit to converting **cp** with **One-Hot Encoding**, let's just see what happens when we convert **cp** without saving the results. This will make it easy to see how `get_dummies()` works.

In [7]:
pd.get_dummies(X, columns=['cp']).head()

,age,sex,restbp,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,cp_1.0,cp_2.0,cp_3.0,cp_4.0
0,63.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,True,False,False,False
1,67.0,1.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,False,False,False,True
2,67.0,1.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,False,False,False,True
3,37.0,1.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,False,False,True,False
4,41.0,0.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,False,True,False,False


As we can see in the printout above, `get_dummies()` puts all of the columns it does not process in the front and it puts **cp** at the end. It also splits **cp** into **4** columns, just like we expected it. **cp_1.0** is `1` for any patient that scored a **1** for chest pain and `0` for all other patients. **cp_2.0** is `1` for any patient that scored **2** for chest pain and `0` for all other patients. **cp_3.0** is `1` for any patient that scored **3** for chest pain and **cp_4.0** is `1` for any patient that scored **4** for chest pain.

Now that we see how `get_dummies()` works, let's use it on the four categorical columns that have more than 2 categories and save the result.



In [8]:
# Create dummy variable for columns "'cp', 'restecg', 'slope', 'thal'" and store it in X_encoded dataframe. Use .head method to display the first 5 rows


,age,sex,restbp,chol,fbs,thalach,exang,oldpeak,ca,cp_1.0,...,restecg_0.0,restecg_1.0,restecg_2.0,slope_1.0,slope_2.0,slope_3.0,thal_0.0,thal_3.0,thal_6.0,thal_7.0
0,63.0,1.0,145.0,233.0,1.0,150.0,0.0,2.3,0.0,True,...,False,False,True,False,False,True,False,False,True,False
1,67.0,1.0,160.0,286.0,0.0,108.0,1.0,1.5,3.0,False,...,False,False,True,False,True,False,False,True,False,False
2,67.0,1.0,120.0,229.0,0.0,129.0,1.0,2.6,2.0,False,...,False,False,True,False,True,False,False,False,False,True
3,37.0,1.0,130.0,250.0,0.0,187.0,0.0,3.5,0.0,False,...,True,False,False,False,False,True,False,True,False,False
4,41.0,0.0,130.0,204.0,0.0,172.0,0.0,1.4,0.0,False,...,False,False,True,True,False,False,False,True,False,False


-----

# Convert all columns to Int, Float or Bool

**XGBoost** requires that all data be either `int`, `float` or `boolean` data types. We can use `dtypes` to see if there are any columns that need to be converted...

In [9]:
X_encoded.dtypes

age            float64
sex            float64
restbp         float64
chol           float64
fbs            float64
thalach        float64
exang          float64
oldpeak        float64
ca             float64
cp_1.0            bool
cp_2.0            bool
cp_3.0            bool
cp_4.0            bool
restecg_0.0       bool
restecg_1.0       bool
restecg_2.0       bool
slope_1.0         bool
slope_2.0         bool
slope_3.0         bool
thal_0.0          bool
thal_3.0          bool
thal_6.0          bool
thal_7.0          bool
dtype: object

Looks good, nothing to do for data types, all data type are either float, integer, or boolean.


----

# Build a Preliminary XGBoost Model
The data is now correctly formatted so to make the **XGBoost** model. Now we simply split the data into **training** and **testing** datasets and build the model.

Let's first check how well or bad the data is balanced. We want to know, if the "0"'s and "1"'s are kind of balanced, or if one or the other is more porminent in the data.

In [10]:
# Calculate proportion (fraction) of class 1 in y, to see what is the proportion of data points where "Heart Disease is 1 (true)"


0.45874587458745875

The answer is, that about 46% of the data is labled to have a heart disease (y = 1), which is pretty balanced. So we don't need to use "stratify" (which helps to make sure we have the minority class), we just can split it in a asimple way.

So create the train and test data using these variable names:
**X_train, X_test, y_train, y_test**

Allow for 15% of data to be used as test, use random_state = 42 for reproducibility

In [11]:
X_train, X_test, y_train, y_test = ...  # create test and training data

Now create a xgboost classifier model with:
- objective='binary:logistic', random_state=42
- Name of the model:  **clf_xgb**

In [12]:
clf_xgb = xgb.XGBClassifier(
    objective='binary:logistic', 
    random_state=42)

Train the created model **clf_xgb** with the training data **(X_train, y_train)**

In [13]:
clf_xgb.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

OK, we've built an **XGBoost** model for classification. Let's see how it performs on the **Testing Dataset** by running the **Testing Dataset** down the model and drawing a **Confusion Matrix**.

In [ ]:
# Make predictions on the test set


# Create confusion matrix



In the confusion matrix, we see that of the **18 + 3 = 21** people that did not have Heart Disease, 18 **(86%)** were correctly classified. And of the **4 + 21 = 25** people that have Heart Disease, 21 **(84%)** were correctly classified. Can we do better?  

One thing we can try is to any of its many hyperparameters. So let's optimize the hyperparameters and see if we can improve the performance.

----

# Optimize Parameters using Cross Validation and GridSearch()

**XGBoost** has a lot of *hyperparameters*, parameters that we have to manual configure and are not determined by XGBoost itself, including `n_estimators` (the number of XGBoost trees to make), `max_depth` (the maximum tree depth), `learning_rate`, (the learning rate, or "eta"), `gamma` (the parameter that encourages pruning), and `reg_lambda` (the regularization parameter lambda). So let's try to find the optimal values for these hyperparameters in hopes that we can improve the accuracy with the **Testing Dataset**.

**NOTE:** Since we have many hyperparameters to optimize, we will use `GridSearchCV()` for hyperparameter tuning. `GridSearchCV()` automatically tests all possible combinations and finds the best ones. We need to specify a set of potential values for these hyperparameters and `GridSearchCV()` will tests all possible combinations.

In [ ]:
# we create a grid of all parameter combinations
param_grid = {
    'max_depth': [3, 4, 5, 6, 7, 8],
    'n_estimators': range(50, 250, 50),
    'learning_rate': [0.1, 0.01, 0.05],
    'gamma': [0, 0.25, 0.5, 1.0],
    'reg_lambda': [0, 1.0, 10.0, 100.0]
}

#for each combination, GridSearch will train multiple XGBoost models using cross-validation, find optimal parameters and create XGBoost model using these new optimal parameters
optimal_xgb = GridSearchCV(
    estimator = xgb.XGBClassifier(objective='binary:logistic',
                                random_state=42),
    param_grid = param_grid,
    verbose = 1, 
    n_jobs = -1,
    cv = 5
)

#retrain the model using optimal parameters.
optimal_xgb.fit(X_train, y_train)
print(optimal_xgb.best_params_)

So, after testing all possible combinations of the potential parameter values with **Cross Validation**, we see that we should set `gamma=0`, `learn_rate=0.1`, `max_depth=6`, `n_estimators=150` and `reg_lambda=10`.  

You can rerun this optimization with `verbose=2` if you want to see individual entries of the parameter optimization process

----

# Building, Evaluating, Drawing, and Interpreting the Optimized XGBoost Model

This is optional (as we already have **optimal_xgb** as our optimized model), but we can rebuild the final **XGBoost** model with previously obtained optimal hyperparameters.

In [16]:
final_model = ...

final_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.25, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

Now let's draw another confusion matrix to see if the optimized **XGBoost** model does better.

In [ ]:
# Make predictions on the test set


# Create confusion matrix



We see that the optimized **XGBoost** model is a little better at classifying people that do not have heart disease and better at classifying people that have it. 

Of the **16 + 5 = 21** people that did not have heart disease, **16 (76%)** were correctly classified. This is worse than before the optimization and may cause false alarms!  
Of the **2 + 23 = 25** people with heart disease, **23 (92%)** were correctly classified. This is better than the preliminary model.

Maybe we want to have more conservative model in healthcare to more detect the disease with more accuracy, at the cost of causing some false alarms.